# Измерение качества моделей

## Метрики качества

Это практическое задание посвящено ознакомлению с инструментами оценки качества моделей машинного обучения, которые предоставляет библиотека `scikit-learn`. Метрики качества, представленные различными функциями, находятся в модуле `sklearn.metrics`. Мы начнем с загрузки набора данных из файла `data.csv` при помощи функции `read_csv` из библиотеки `pandas`. Этот набор данных содержит информацию о предсказаниях различных алгоритмов машинного обучения для решения задачи классификации. Колонка `prediction` - это результаты работы одного из этих алгоритмов. Целевая переменная содержится в столбце `target` (класс 0 или 1). Подсчитайте значение `true negative`, `false negative`, `true positive` и `false positive`. Запишите эти значения через запятую, сохраняя приведенный порядок, в переменную `answer1`, которая будет являтся строкой. Далее, посчитайте для этих данных значение таких метрик как `precision`, `recall` и `f1 score` с точностью до двух знаков после запятой. Запишите результаты в строго заданном порядке через запятую в переменную `answer2`, которая так же будет являться строкой

### *РЕШЕНИЕ*

In [24]:
import pandas as pd
from sklearn.metrics import confusion_matrix

data = pd.read_csv('data.csv')
mtr=confusion_matrix(data['target'],data['prediction'])
answer1 = [mtr[0][0],mtr[1][0],mtr[1][1], mtr[0][1]]
answer1 = ','.join(map(str,answer1))
answer1

'63,4,114,7'

mtr

In [17]:
from sklearn.metrics import precision_score, recall_score, f1_score
def get_score(estimator):
    score=estimator(data.target,data.prediction)
    return str(round(score,2))
answer2 = ','.join(map(get_score, [precision_score, recall_score, f1_score]))
answer2

'0.94,0.97,0.95'

В столбцах `scores_1` и `scores_2` содержаться оценки вероятности пренадлежности объектов к классу 1 для двух разных алгоритмов машинного обучения. Рассчитайте площадь под ROC-кривой для каждого алгоритма и сравните их. В качестве ответа `answer3` приведите большее из двух значений, округленное до трех знаков после запятой.

### *РЕШЕНИЕ*

In [22]:
from sklearn.metrics import roc_auc_score

score_1 = roc_auc_score(data['target'], data['scores_1'])
score_2 = roc_auc_score(data['target'], data['scores_2'])
print("score 1 > score 2 ?{} ".format(score_1 > score_2))
answer3 = max(score_1, score_2)
answer3 = round(answer3, 3)
answer3


score 1 > score 2 ?True 


0.993

## Метод скользящего контроля

Во второй части данного практического задания мы изучать различные методы оценки моделей машинного обучения. Загрузите набор данных `Breast Cancer Wisconsin (Diagnostic)`, используя функцию `load_breast_cancer` из модуля `sklearn.datasets`. Этот датасет позволяет решать задачу предсказания рака груди по различным характеристикам опухоли. В данном случае, целевая переменная принимает два значения, соответствующие доброкачественной и злокачественной опухоли. Проверьте, является ли данная выборка сбалансированной.

In [26]:
from sklearn.datasets import load_breast_cancer

X, y = load_breast_cancer(return_X_y=True)
counts = pd.value_counts(y)
print("Is this a balanced dataset? {}".format(counts[1] == counts[0]))

Is this a balanced dataset? False


Первый метод, который мы будем использовать, - это случайное разбиение датасета на тренировочную и тестовую выборку с помощью функции `train_test_split` из `sklearn.model_selection` с параметрами `random_state=3` и `test_size=0.33`. Если выборка является несбалансированной передайте целевую переменную в эту функцию в качестве аргумента `stratify`.

Обучите логистическую регрессию (класс `LogisticRegression` из модуля `sklearn.linear_model`) с параметром конструктора `random_state=42` и метод K ближайших соседей (класс `KNeighborsClassifier` из модуля `sklearn.neighbors`) на тренировочной выборке. Оцените качество на тестовой выборке для каждой из моделей. В качестве метрики качества используйте `recall`. Какая из моделей показывает лучший результат? Ответом на это задание `answer4` является этот результат, округленный до трех знаков после запятой.

### *РЕШЕНИЕ*

In [85]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import recall_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression

X_train, X_test, y_train, y_test = train_test_split(X,y, random_state=3, test_size=0.33, stratify=y)

def get_score(model_cls,**kwargs):
    model=model_cls(**kwargs)
    model.fit(X_train,y_train)
    y_pred=model.predict(X_test)
    score=recall_score(y_test,y_pred)
    return score

scoreKN=get_score(KNeighborsClassifier)
scoreReg=get_score(LogisticRegression,random_state=42)

def best_score(scoreKN, scoreReg):
    if scoreKN>scoreReg:
        answer=round(scoreKN,3)
        name="K neighbours"
    else:
        answer=round(scoreReg,3)
        name="Logistic Regression"
    return answer,name

answer4,name4=best_score(scoreKN, scoreReg)
print("name: "+name4+"\n"+str(answer4))



name: K neighbours
0.975


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.9661016949152542

Далее мы проведем оценку каждой из этих моделей в соответствии с методом скользящего контроля с помощью функции `cross_val_score` из модуля `sklearn.model_selection`. В качестве параметра кросс-валидации `cv` в этой функции используйте экземпляр класса `StratifiedKFold` из `sklearn.model_selection` с тремя разбиениями. 

Функция `cross_val_score` возвращает количество оценок, соответствующие числу разбиений.
В качестве итогового результата используете среднее значение полученных оценок с помощью метрики `recall`. Какая модель работает лучше в это случае? Какие выводы можно из этого сделать? Ответом на это задание `answer5` является лучший итоговый результат, округленный до трех знаков после запятой.

### *РЕШЕНИЕ*

In [91]:
from sklearn.model_selection import cross_val_score, StratifiedKFold

def score_(model_cls,**kwargs):
    model=model_cls(**kwargs)
    cv=StratifiedKFold(3)
    score=cross_val_score(model,X,y,cv=cv,scoring='recall')
    return score.mean()

score1=score_(LogisticRegression,random_state=42)
score2=score_(KNeighborsClassifier)

def get_best_score(score2, score1):
    if score2>score1:
        answer=round(score2,3)
        name="K neighbours"
    else:
        answer=round(score1,3)
        name="Logistic Regression"
    return answer,name

answer5, name5 = get_best_score(score2,score1)
print("name: "+name5+"\n"+str(answer5))

name: Logistic Regression
0.964


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_

In [67]:
def estimate_model(model_cls, **kwargs):
    model = model_cls(**kwargs)
    cv = StratifiedKFold(3)
    scores = cross_val_score(model, X, y, cv=cv, scoring='recall')
    return scores.mean()

score_knn = estimate_model(KNeighborsClassifier)
score_logistic_regression = estimate_model(LogisticRegression, random_state=42)

model_name, answer5 = get_best_score(score_knn, score_logistic_regression)
print("best score: %s" % model_name)

best score: Logistic Regression


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_

# Строка с ответами

In [ ]:
output = """TN,FN,TP,FP = {0}
Precision,Recall,F1 Score = {1}
Best ROC AUC Score {2:.3f}
Random Split {3:.3f}
Cross Val Score {4:.3f}"""
print(output.format(answer1, answer2, answer3, answer4, answer5))